<a href="https://colab.research.google.com/github/BillyMuthiani/Ultra_fast-Book_summarizer/blob/main/Book_Recommendor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.6 MB/s eta 0:00:00


In [ ]:

df=pd.read_csv('books.csv',on_bad_lines='skip')
df.head(31)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown


In [ ]:
from os import read
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from rapidfuzz import process, fuzz


class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None): return self
    def transform(self, X): return X[self.key].fillna("")

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, keys): self.keys = keys
    def fit(self, X, y=None): return self
    def transform(self, X): return X[self.keys].fillna(0)

class BookRecommender:
    def __init__(self, n_neighbors=10, fuzzy_threshold=75):
        self.n_neighbors = n_neighbors
        self.fuzzy_threshold = fuzzy_threshold
        self.model = None
        self.indices = None

    def fit(self, books_df):
        df = books_df.copy()
        df.reset_index(drop=True, inplace=True)
        df['text_features'] = df['title'] + ' ' + df['authors'].fillna("")
        text_pipeline = Pipeline([
            ('selector', TextSelector('text_features')),
            ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000))
        ])
        numeric_cols = ['average_rating', 'ratings_count', 'text_reviews_count', 'num_pages']
        num_pipeline = Pipeline([
            ('selector', NumberSelector(numeric_cols)),
            ('scaler', StandardScaler())
        ])
        full_pipeline = FeatureUnion([('text', text_pipeline), ('numeric', num_pipeline)])
        features = full_pipeline.fit_transform(df)
        self.model = NearestNeighbors(n_neighbors=self.n_neighbors, metric='cosine', algorithm='brute')
        self.model.fit(features)
        self.indices = pd.Series(df.index, index=df['title'])
        self.books_df = df
        self.feature_pipeline = full_pipeline
        return self

    def _find_closest_title(self, title):
        titles = list(self.indices.index)
        match, score, _ = process.extractOne(title, titles, scorer=fuzz.WRatio)
        return (match, score)

    def recommend(self, title, n_recommendations=5, auto_accept_closest=True):
        if title not in self.indices:
            closest, score = self._find_closest_title(title)
            if score >= self.fuzzy_threshold and auto_accept_closest:
                print(f"Using closest match: '{closest}' (score: {score})")
                title = closest
            else:
                raise ValueError(f"Book title '{title}' not found. Closest match '{closest}' (score: {score}).")
        idx = self.indices[title]
        if isinstance(idx, pd.Series): # Check if multiple indices are returned
            idx = idx.iloc[0] # Get the first index if multiple are returned
        vec = self.feature_pipeline.transform(self.books_df.iloc[idx:idx+1]) # Select row as DataFrame slice
        distances, indices = self.model.kneighbors(vec, n_neighbors=n_recommendations+1)
        rec_idxs = indices.flatten()[1:]
        return self.books_df.iloc[rec_idxs][['title', 'authors', 'average_rating']].reset_index(drop=True)


if __name__ == "__main__":
    df = pd.read_csv('books.csv',on_bad_lines='skip')
    df = df.rename(columns=lambda x: x.strip())
    recommender = BookRecommender(n_neighbors=10)
    recommender.fit(df)
    book = input("Enter a book title: ")
    print(f"Recommendations for '{book}':")
    print(recommender.recommend(book))
    print("Recommender system is ready to use.")

Enter a book title: Hachet
Recommendations for 'Hachet':
Using closest match: 'Hatchet (Brian's Saga  #1)' (score: 75.00000000000001)
                 title                                   authors  \
0      Eat  Pray  Love                         Elizabeth Gilbert   
1  Water for Elephants                                Sara Gruen   
2             The Road                           Cormac McCarthy   
3        The Alchemist  Paulo Coelho/Alan R. Clarke/Özdemir İnce   
4      Never Let Me Go                            Kazuo Ishiguro   

   average_rating  
0            3.55  
1            4.09  
2            3.97  
3            3.86  
4            3.82  
Recommender system is ready to use.
